In [23]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData-master/DefectData/inst/extdata/terapromise/mccabe/PC2.arff') 
df = pd.DataFrame(data[0])

In [24]:
df.shape

(745, 37)

In [25]:
df.head()

,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,DESIGN_COMPLEXITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,3.0,2.0,2.0,5.0,4.0,2.0,0.67,2.0,2.0,2.0,...,6.0,0.20,2.0,7.0,2.0,6.0,10.0,87.50,3.0,b'N'
1,3.0,1.0,0.0,0.0,4.0,2.0,1.00,2.0,2.0,2.0,...,5.0,2.00,2.0,8.0,2.0,8.0,1.0,0.00,0.0,b'N'
2,7.0,5.0,10.0,1.0,12.0,4.0,0.31,6.0,2.0,4.0,...,23.0,0.25,26.0,49.0,9.0,17.0,16.0,78.57,13.0,b'N'
3,7.0,4.0,9.0,1.0,12.0,4.0,0.33,6.0,2.0,4.0,...,18.0,0.24,26.0,37.0,13.0,17.0,17.0,76.92,12.0,b'N'
4,7.0,2.0,21.0,1.0,12.0,4.0,0.16,6.0,2.0,2.0,...,13.0,0.14,40.0,46.0,14.0,15.0,29.0,84.62,25.0,b'N'


In [32]:
buggy = b'Y'
clean = b'N'
x = df.drop(['Defective'],axis = 'columns')

for i in range(745):
    if df.iloc[i,36] == buggy:
        df.iloc[i,36] = 1
    else:
        df.iloc[i,36] = 0

y = df['Defective']
y=y.astype(int)
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Defective, dtype: int32

In [34]:
from sklearn.cluster import DBSCAN
dbs=DBSCAN(eps=0.3, min_samples=10)
dbs.fit(x)
dbs_pred=dbs.labels_
dbs_pred[:121]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1], dtype=int64)

In [96]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy = 'minority')
x, y = smote.fit_resample(x,y)

print(y.value_counts())

0    28
1    28
Name: defects, dtype: int64


In [129]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.4,stratify = y)

len(x_train)

33

In [130]:
y_train.value_counts()

1    17
0    16
Name: defects, dtype: int64

In [131]:
y_test.value_counts()

0    12
1    11
Name: defects, dtype: int64

In [100]:
# training the model on training set
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)

# making predictions on the testing set
y_pred = gnb.predict(x_test)

In [101]:
from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test,y_pred)
TP = conf[0,0]
FP = conf[0,1]
TN = conf[1,0]
FN = conf[1,1]

#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.2041241452319315
Accuracy : 0.5217391304347826
F-Score : 0.6666666666666666


In [102]:
from sklearn.svm import SVC

svc_model = SVC() 
svc_model.fit(x_train, y_train) 

y_pred = svc_model .predict(x_test) 


from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test,y_pred)
TP = conf[0,0]
FP = conf[0,1]
TN = conf[1,0]
FN = conf[1,1]

#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.37080992435478316
Accuracy : 0.6086956521739131
F-Score : 0.7096774193548387


In [106]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)


from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test,y_pred)
TP = conf[0,0]
FP = conf[0,1]
TN = conf[1,0]
FN = conf[1,1]

#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.2041241452319315
Accuracy : 0.5217391304347826
F-Score : 0.6666666666666666


In [112]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
rf= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
rf.fit(x_train, y_train)  

y_pred= rf.predict(x_test)  

from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test,y_pred)
TP = conf[0,0]
FP = conf[0,1]
TN = conf[1,0]
FN = conf[1,1]

#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.2954684201426394
Accuracy : 0.5652173913043478
F-Score : 0.6875000000000001


In [115]:
#Fitting Logistic Regression to the training set  
from sklearn.linear_model import LogisticRegression  
log= LogisticRegression(random_state=0)  
log.fit(x_train, y_train) 

y_pred= log.predict(x_test)  

from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test,y_pred)
TP = conf[0,0]
FP = conf[0,1]
TN = conf[1,0]
FN = conf[1,1]

#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : -0.3223291856101521
Accuracy : 0.391304347826087
F-Score : 0.5625


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [134]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

estimator = []
estimator.append(('LR', log))
estimator.append(('KN', knn))
estimator.append(('R', rf))
  
# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(x_train, y_train)
y_pred = vot_hard.predict(x_test)

from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test,y_pred)
TP = conf[0,0]
FP = conf[0,1]
TN = conf[1,0]
FN = conf[1,1]

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

# using accuracy_score metric to predict accuracy
# score = accuracy_score(y_test, y_pred)
# print("Hard Voting Score % d" % score)

MCC : -0.11236664374387369
Accuracy : 0.4782608695652174
F-Score : 0.625


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
